In [3]:
# tokenization 
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8") # raw bytes


In [24]:
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(text)
print("length:", len(text))
print(len(text.strip()))

print('--machine readable tokens is utf encoding so the no of bytes read by the machines is more')
print(tokens)
print("length:", len(tokens))

# we can decode back to text using the tokens 
decoded = bytes(tokens).decode("utf-8")
print(decoded)
print(len(decoded))
print(decoded == text)

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
533
--machine readable tokens is utf encoding so the no of bytes read by the machines is more
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135

In [124]:
# BPE algorithm 
# it finds the most frequent pair from get stats 
# assign it a new token id 
# merge it into vocab 
# repeat this N times 

# lets write the steps 
# find the most freq pair 
def stats(tokens):
    # given tokens as a list of all the possible tokens find out the most freq token or get the stats 
    counts = {}
    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0 )+1 
    return counts 
# # get the top pair 
# stats_tokens = stats(tokens)
# top_pair = max(stats_tokens, key = stats_tokens.get)
# # now merge this toppair to the vocab, do we remove the individual pair elements --no ??
def merge(tokens, new_pair, idx  ):
    newids = []
    # this idx will be the max(tokens)+1 
    # idx = max(tokens)+1
    # # each new merge will have a new idx 
    i = 0 
    while i < len(tokens):
        if i < len(tokens) - 1  and  tokens[i]==new_pair[0] and tokens[i+1]==new_pair[1]:
            # we skip the occurence of the new_pair and instead just keep the occurence of this pair in the string here 
            newids.append(idx)
            i+=2 
        else:
            newids.append(tokens[i])
            i+=1
    return newids
def train_bpe(iterations, text):

    merges = []    
    id_to_pair = {}
    # lets say we have the tokens as the initial tokens 
    tokens = list(text.encode("utf-8"))
    # we want to build the BPE for this by iterating for 10 steps 
    for _ in range(iterations):
        # print("we start with total vocab ", len(tokens))
        freq_tokens = stats(tokens)
        # get the max token here 
        top_pair = max(freq_tokens, key = freq_tokens.get)
        print(id_to_pair)
        new_id = max(tokens)+1
        merges.append((top_pair, new_id))
        id_to_pair[new_id]= top_pair
        # now we get the max id and the numebr of times this merge hapeps 
        # print("ids replaced:", top_pair)
        # print("coutn by which our vocab decreases", freq_tokens[top_pair])
        # print("The “+1” you were trying to add belongs to vocabulary size, not sequence length")
        tokens = merge(tokens, top_pair, new_id )
        # tokens here is the training text encoded under the learned merges
    return tokens, merges, id_to_pair


In [125]:
final_vocab, merges, id_to_pair =   train_bpe(10, text)
# lets get to decoding now 
# now we have a new tokeniser , we can go ahead and encode and decode with this BPE tokenizer now 
# merge(tokens, top_pair, new_id )
# freq tokens is the list for pair and new_id added to this vocab 
def encode_bpe(merges , text):
    tokens = list(text.encode("utf-8"))
    for pair, id in merges: # we track all the merges in the training order 
        tokens = merge(tokens, pair, id )
    return tokens
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
encoded_text = encode_bpe(merges, text)
print("original text encoding:", len(text.encode("utf-8")))
print(len(encoded_text))

{}
{241: (101, 32)}
{241: (101, 32), 242: (240, 159)}
{241: (101, 32), 242: (240, 159), 243: (226, 128)}
{241: (101, 32), 242: (240, 159), 243: (226, 128), 244: (105, 110)}
{241: (101, 32), 242: (240, 159), 243: (226, 128), 244: (105, 110), 245: (115, 32)}
{241: (101, 32), 242: (240, 159), 243: (226, 128), 244: (105, 110), 245: (115, 32), 246: (97, 110)}
{241: (101, 32), 242: (240, 159), 243: (226, 128), 244: (105, 110), 245: (115, 32), 246: (97, 110), 247: (116, 104)}
{241: (101, 32), 242: (240, 159), 243: (226, 128), 244: (105, 110), 245: (115, 32), 246: (97, 110), 247: (116, 104), 248: (242, 133)}
{241: (101, 32), 242: (240, 159), 243: (226, 128), 244: (105, 110), 245: (115, 32), 246: (97, 110), 247: (116, 104), 248: (242, 133), 249: (242, 135)}
original text encoding: 616
508


In [135]:
#lets decode now 

def decode_bpe(tokens, id_to_pair):
    # Keep expanding learned tokens until everything is a base byte (<256)
    for new_id in sorted(id_to_pair.keys(), reverse=True):
        pair = id_to_pair[new_id]
        new_tokens = []
        i = 0
        while i < len(tokens):
            if tokens[i] == new_id:
                a, b = pair
                new_tokens.extend([a, b])
                i += 1
            else:
                new_tokens.append(tokens[i])
                i += 1
        tokens = new_tokens

    # At this point, tokens are all raw byte values
    return tokens

out = decode_bpe(tokens,id_to_pair )
print(len(out)==len(text.encode("utf-8")))



True
